# 추천 시스템 (Recommender System)

컨텐츠 기반 필터링 (content-based filtering) - 사용자의 이전 행동 명시적인 피드백을 통해서 사용자가 좋아하는 것인가 유사한가를 고려해서 추천

협업 필터링 (collarborativefiltering) - 사용자와 항목 간의 유사성을 동시에 사용해 추천

### Surprise
- 추천 시스템 개발을 위한 라이브러리
- 다양한 모델과 데이터 제공
- scikit-learn과 유사한 사용 방법

간단한 surprise 실습

In [33]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [34]:
data = Dataset.load_builtin("ml-100k", prompt = False)
data.raw_ratings[:10] 

# 사용자가 어떤 영화에 대해서 rating을 몇 점을 줬는지가 중요한 데이터
# 유저 / 아이템(영화) / rating 점수 / 아이디

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [35]:
model = SVD()

In [36]:
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9325  0.9456  0.9363  0.9310  0.9359  0.9363  0.0051  
MAE (testset)     0.7354  0.7440  0.7398  0.7338  0.7347  0.7375  0.0038  
Fit time          3.47    3.33    3.41    3.38    3.56    3.43    0.08    
Test time         0.11    0.15    0.09    0.15    0.10    0.12    0.02    


{'test_rmse': array([0.9325111 , 0.94561794, 0.93634852, 0.93102107, 0.93588652]),
 'test_mae': array([0.73535181, 0.74400888, 0.73981926, 0.73379462, 0.73467196]),
 'fit_time': (3.4724106788635254,
  3.3281872272491455,
  3.405137538909912,
  3.379066228866577,
  3.5570147037506104),
 'test_time': (0.11387085914611816,
  0.15181875228881836,
  0.0937497615814209,
  0.1455061435699463,
  0.09778571128845215)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

- 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
    - ex) 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천

- 유사도를 기반으로 추천

- 장단점
    - 장점
        - 많은 수의 사용자를 대상으로 쉽게 확장 가능
        - 사용자가 관심을 갖지 않던 상품 추천 가능
    - 단점
        - 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
        - 사용자의 기존 관심사항을 기반으로만 추천 가능

In [37]:
import numpy as np
from surprise import Dataset

- 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
- 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [38]:
data = Dataset.load_builtin('ml-100k', prompt = False) # 영화
raw_data = np.array(data.raw_ratings, dtype = int)

In [39]:
# 0부터 시작할 수 있도록
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [40]:
# 인접 행렬 크기 구하기
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])

# 왜 + 1 하는 거지
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [41]:
# 인접 행렬 만들기 - 특정 값에 1을 넣어줌(1이 있는 곳이 데이터가 있는 것)
# 관람의 유무
adj_matrix = np.ndarray(shape, dtype = int)

for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1.

adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [42]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 183, Best Match ID: 275


In [43]:
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

- 유클리드 거리를 사용해 추천

- 거리가 가까울수록(값이 작을수록) 나와 유사한 사용자

In [44]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 14.832396974191326, Best Match ID: 737


In [45]:
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0:
        recommend_list.append(i)

print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


- 코사인 유사도를 사용해 추천

- 두 벡터가 이루고 있는 각을 계산

In [46]:
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [47]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.5278586163659506, Best Match ID: 915


In [48]:
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0:
        recommend_list.append(i)

print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [49]:
# 평가 점수
adj_matrix = np.ndarray(shape, dtype=int)

for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [50]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 55.06359959174482, Best Match ID: 737


In [51]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.569065731527988, Best Match ID: 915


# 협업 필터링 (Collaborative Filtering)
    - 사용자와 항목의 유사성을 동시에 고려해 추천
    - 기존에 내 관심사가 아닌 항목이라도 추천 가능
    - 자동으로 임베딩 학습 가능
    
    - 장단점
        - 장점
            - 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요없다
            - 기존의 관심사가 아니더라도 추천 가능

        - 단점
            - 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
            - 추가 특성을 사용하기 어려움

In [52]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [53]:
data = Dataset.load_builtin('ml-100k', prompt = False)

KNN을 사용한 협업 필터링

In [54]:
model = KNNBasic()
cross_validate(model, data, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9786  0.9780  0.9769  0.9788  0.9788  0.9782  0.0007  
MSE (testset)     0.9577  0.9565  0.9543  0.9581  0.9581  0.9569  0.0015  
Fit time          0.62    0.58    0.55    0.51    0.37    0.53    0.09    
Test time         2.21    2.20    2.11    2.16    1.89    2.11    0.12    


{'test_rmse': array([0.97860847, 0.97799451, 0.97686985, 0.97882905, 0.97884238]),
 'test_mse': array([0.95767453, 0.95647327, 0.95427471, 0.9581063 , 0.9581324 ]),
 'fit_time': (0.6208491325378418,
  0.578953742980957,
  0.5475592613220215,
  0.5106585025787354,
  0.37221550941467285),
 'test_time': (2.205608367919922,
  2.198300838470459,
  2.1079869270324707,
  2.163285970687866,
  1.8882200717926025)}

SVD를 사용한 협업 필터링

In [55]:
model = SVD()
cross_validate(model, data, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9360  0.9420  0.9267  0.9309  0.9326  0.9336  0.0052  
MSE (testset)     0.8762  0.8873  0.8587  0.8665  0.8697  0.8717  0.0096  
Fit time          3.66    3.69    3.75    3.79    3.12    3.60    0.24    
Test time         0.14    0.15    0.14    0.12    0.11    0.13    0.01    


{'test_rmse': array([0.93604857, 0.94198054, 0.92667048, 0.93087543, 0.93259262]),
 'test_mse': array([0.87618692, 0.88732734, 0.85871818, 0.86652907, 0.869729  ]),
 'fit_time': (3.657914876937866,
  3.6933929920196533,
  3.7472572326660156,
  3.790456533432007,
  3.1232247352600098),
 'test_time': (0.13521909713745117,
  0.14861154556274414,
  0.1369178295135498,
  0.11896324157714844,
  0.1115407943725586)}

NMF를 사용한 협업 필터링

In [56]:
model = NMF()
cross_validate(model, data, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9639  0.9625  0.9658  0.9587  0.9665  0.9635  0.0028  
MSE (testset)     0.9291  0.9263  0.9327  0.9191  0.9342  0.9283  0.0053  
Fit time          3.67    3.78    3.86    3.90    3.65    3.77    0.10    
Test time         0.10    0.10    0.09    0.10    0.09    0.10    0.00    


{'test_rmse': array([0.96390307, 0.96245984, 0.96577583, 0.95872306, 0.96652027]),
 'test_mse': array([0.92910913, 0.92632894, 0.93272295, 0.91914991, 0.93416143]),
 'fit_time': (3.6671571731567383,
  3.7783453464508057,
  3.85697078704834,
  3.9011683464050293,
  3.652040481567383),
 'test_time': (0.0999748706817627,
  0.10198378562927246,
  0.09498810768127441,
  0.09897637367248535,
  0.0910031795501709)}

SVD++를 사용한 협업 필터링

In [ ]:
# 출력하는데 오래 걸림
model = SVDpp()
cross_validate(model, data, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

하이브리드(Hybrid)
- 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
- 많은 하이브리드 방식이 존재
- 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [77]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [78]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [79]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [80]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating

In [81]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [82]:
U, S, V  = randomized_svd(adj_matrix, n_components = 2)
S = np.diag(S)

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:369: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


In [83]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [84]:
np.matmul(np.matmul(U, S), V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

- 사용자 기반 추천
- 나와 비슷한 취향을 가진 다른 사용자의 행동 추천
- 사용자 특징 벡터의 유사도 사용

In [85]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999942295956324, Best Match ID: 235


In [86]:
recommend_list = []

for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


- 항목 기반 추천
- 내가 본 항목과 비슷한 항목을 추천
- 항목 특징 벡터의 유사도 사용

In [87]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999951364144, Best Match ID: 1287


In [88]:
recommend_list = []

for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)

print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

비음수 행렬 분해를 사용한 하이브리드 추천

In [89]:
A, B, iter = non_negative_factorization(adj_matrix, n_components = 2)

In [90]:
np.matmul(A, B)

array([[3.71107433e+00, 1.48461856e+00, 7.39541570e-01, ...,
        3.64501983e-03, 1.45513751e-02, 1.44116215e-02],
       [2.11729713e+00, 2.37145679e-01, 5.51637757e-01, ...,
        4.76290749e-03, 2.84605930e-05, 0.00000000e+00],
       [9.85325089e-01, 1.10360320e-01, 2.56715279e-01, ...,
        2.21651094e-03, 1.32446863e-05, 0.00000000e+00],
       ...,
       [1.04478344e+00, 1.17019891e-01, 2.72206478e-01, ...,
        2.35026384e-03, 1.40439223e-05, 0.00000000e+00],
       [1.45769331e+00, 5.42108391e-01, 2.99217251e-01, ...,
        1.61232500e-03, 5.15892655e-03, 5.10748255e-03],
       [2.44709957e+00, 9.41278705e-01, 4.95671746e-01, ...,
        2.56934867e-03, 9.08400301e-03, 8.99501717e-03]])

사용자 기반 추천

In [91]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999942295956324, Best Match ID: 235


In [92]:
recommend_list = []

for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


항목 기반 추천

In [93]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999951364144, Best Match ID: 1287


In [94]:
recommend_list = []

for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)

print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 